ASAP CRN Metadata validation 

# Team Hafler. ASAP CRN Metadata validation

9 Oct 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, intervention_typer, create_upload_medadata_package
from checksums import get_md5_hashes, authenticate_with_service_account

%load_ext autoreload
%autoreload 2

root_path = Path.home() / ("Projects/ASAP/data/teams")


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


> SANITY CHECK: verify reading from google doc works.

```python
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")
```


## Load original tables 
These were submitted as v1.0


Team Hafler


In [3]:
## convert 
team = "hafler"
dataset_name = "sn-rnaseq-pfc"


metadata_path = root_path / f"{team}/{dataset_name}/metadata"

sheets = ["SAMPLE","SUBJECT","CLINPATH","STUDY","PROTOCOL"]
excel_path = metadata_path / "og/ASAP_CDE_ALL_Team_Hafler_v1.xlsx"
STUDY = pd.read_excel(excel_path,sheet_name="STUDY",header=1).drop(columns="Field")
CLINPATH = pd.read_excel(excel_path,sheet_name="CLINPATH",header=1).drop(columns="Field")
SUBJECT = pd.read_excel(excel_path,sheet_name="SUBJECT",header=1).drop(columns="Field")
SAMPLE = pd.read_excel(excel_path,sheet_name="SAMPLE",header=1).drop(columns="Field")
PROTOCOL = pd.read_excel(excel_path,sheet_name="PROTOCOL",header=1).drop(columns="Field")
metadata_path

PosixPath('/Users/ergonyc/Projects/ASAP/data/teams/hafler/sn-rnaseq-pfc/metadata')

In [4]:

def add_hafler_batch(sample_df):

    # First batch: HSDG07HC HSDG10HC HSDG148PD HSDG199PD
    # batch[batch.sample_id in ['hSDG07HC', 'hSDG10HC', 'hSDG148PD', 'hSDG199PD']]=1
    Batch_1 = ['hSDG07', 'hSDG10', 'hSDG148', 'hSDG199'] 
    # Second batch: hsDG101HC hsDG13HC hsDG151PD hsDG197PD hsDG30HC hsDG99HC
    Batch_2 = ['hSDG101', 'hSDG13', 'hSDG151', 'hSDG197', 'hSDG30', 'hSDG99']
    # Third batch: hsDG142PD hsDG208PD
    Batch_3 = ['hSDG142', 'hSDG208'] 


    batch_col = []
    for row in sample_df.sample_id:
        if row in Batch_1:
            batch_col.append("Batch_1")
        elif row in Batch_2:
            batch_col.append("Batch_2")
        elif row in Batch_3:
            batch_col.append("Batch_3")
        else:
            print("ERROR >>>>>>>> not no batch info")
            batch_col.append("")


    sample_df['batch'] = batch_col
    return sample_df

SAMPLE = add_hafler_batch(SAMPLE)

In [5]:
# fix replicate & replicate_count
SAMPLE.replace("Nan", "", inplace=True)

SAMPLE['replicate'] = "Rep1"
SAMPLE['replicate_count'] = 1

SAMPLE.head()

,sample_id,source_sample_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hSDG07,hSDG07,HC01,Rep1,1,0,Batch_1,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hSDG101,hSDG101,HC03,Rep1,1,0,Batch_2,Brain,Prefrontal Cortex,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

STUDY = prep_table(STUDY, CDEv1)
STUDY['metadata_version_date']
STUDY.rename(columns={"submittor_email":"submitter_email"}, inplace=True)

In [7]:
SUBJECT.replace("Nan", "", inplace=True)



In [8]:

SUBJECT = prep_table(SUBJECT, CDEv1)


In [9]:
SUBJECT['sex'] = SUBJECT['sex'].replace({'F':"Female", 'M':"Male"})
SUBJECT['race'] = SUBJECT['race'].replace({'W':"White", 'B':"Black or African American"})

SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace({'Normal control':"Healthy Control", "Idiopathic Parkinson's disease":"Idiopathic PD"})


In [10]:
# they "race" was reported under ethnicity, so code as "Not Reported"
SUBJECT['ethnicity'] = 'Not Reported'

In [11]:
SUBJECT[['subject_id', 'source_subject_id']]

,subject_id,source_subject_id
0,HC01,hSDG07
1,HC02,hSDG13
2,HC03,hSDG101
3,HC04,hSDG10
4,HC05,hSDG30
5,HC06,hSDG99
6,PD01,hSDG151
7,PD02,hSDG197
8,PD03,hSDG208
9,PD04,hSDG142


In [12]:
SAMPLE = prep_table(SAMPLE,  CDEv1)


In [13]:
# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

# set time == 0 for all samples
SAMPLE['time'] = 0

SAMPLE['file_type'] = SAMPLE['file_type'].replace({"Fastq":"fastq"})


In [14]:

# need to join with subject to get "sex" and convert to ontology term
SAMPLE_SUBJECT = SAMPLE.merge(SUBJECT, on='subject_id',  how='left')
SAMPLE_og = SAMPLE.copy()
SAMPLE['sex_ontology_term_id'] = SAMPLE_SUBJECT['sex'].replace({"Male":"PATO:0000384 (male)", "Female":"PATO:0000383 (female)" })

# ignore development_stage_ontology_term_id, self_reported_ethnicity_ontology_term_id, assay_ontology_term_id, etc for now. (Check wiht Le)

In [15]:
# fix assay
SAMPLE['assay'] = SAMPLE['assay'].replace({'v3.1 - Single Index, 10x Genomics ':"v3.1 - Single Index"})
# fix assay
SAMPLE['sequencing_length'] = SAMPLE['sequencing_length'].replace({'150bp x2':"150"})

# fix DV200 
SAMPLE.rename(columns={"DV2000":"DV200"}, inplace=True)
#TODO: add file_description I1, R1, R2  "Raw sequencing data"

In [16]:
SAMPLE['input_cell_count'] = 8000

In [17]:
SAMPLE[['sample_id','subject_id','source_sample_id','file_name','file_MD5']]

,sample_id,subject_id,source_sample_id,file_name,file_MD5
0,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_I1_001.fastq.gz,37912362469bc61a936dcc39a2b75138
1,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_R1_001.fastq.gz,b4d94bd92286d318de2b013ea44c5246
2,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_R2_001.fastq.gz,2a3c2c2303948f15d74b89ae4cbff409
3,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_I1_001.fastq.gz,1fbdbaba5e604aa852b9dc6af9e3dea4
4,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_R1_001.fastq.gz,3e14c63f5c55b4f29fcb0b1af5992924
5,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_R2_001.fastq.gz,f34b13c20fd03611b5772037bac11b37
6,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_I1_001.fastq.gz,7649f0db658f8d697b82a0ddc2dfedad
7,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_R1_001.fastq.gz,309aa23c50a13bb99ad88cdc4579d77d
8,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_R2_001.fastq.gz,187af38f53ba8b760e1da2e3fe602051
9,hSDG13,HC02,hSDG13,hsDG13HC_HHT_S1_L002_I1_001.fastq.gz,d3b2e5364b57bfa4a1f50166a9043949


In [18]:
# clean MD5 hashes
SAMPLE['file_MD5'] = SAMPLE['file_MD5'].str.strip()
SAMPLE[['sample_id','subject_id','source_sample_id','file_name','file_MD5']]

,sample_id,subject_id,source_sample_id,file_name,file_MD5
0,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_I1_001.fastq.gz,37912362469bc61a936dcc39a2b75138
1,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_R1_001.fastq.gz,b4d94bd92286d318de2b013ea44c5246
2,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_R2_001.fastq.gz,2a3c2c2303948f15d74b89ae4cbff409
3,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_I1_001.fastq.gz,1fbdbaba5e604aa852b9dc6af9e3dea4
4,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_R1_001.fastq.gz,3e14c63f5c55b4f29fcb0b1af5992924
5,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_R2_001.fastq.gz,f34b13c20fd03611b5772037bac11b37
6,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_I1_001.fastq.gz,7649f0db658f8d697b82a0ddc2dfedad
7,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_R1_001.fastq.gz,309aa23c50a13bb99ad88cdc4579d77d
8,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_R2_001.fastq.gz,187af38f53ba8b760e1da2e3fe602051
9,hSDG13,HC02,hSDG13,hsDG13HC_HHT_S1_L002_I1_001.fastq.gz,d3b2e5364b57bfa4a1f50166a9043949


In [19]:
# the CLINPATH has sample_id as a subject_id, so we need to fix that
subj_samp_mapper = dict(zip( SAMPLE['subject_id'], SAMPLE['sample_id']))
subj_samp_mapper

{'HC01': 'hSDG07',
 'HC03': 'hSDG101',
 'HC04': 'hSDG10',
 'HC02': 'hSDG13',
 'PD04': 'hSDG142',
 'PD05': 'hSDG148',
 'PD01': 'hSDG151',
 'PD02': 'hSDG197',
 'PD06': 'hSDG199',
 'PD03': 'hSDG208',
 'HC05': 'hSDG30',
 'HC06': 'hSDG99'}

In [20]:
CLINPATH = prep_table(CLINPATH, CDEv1)
CLINPATH.rename(columns={"path_thai":"path_thal"}, inplace=True)
CLINPATH.replace("Nan", "", inplace=True)


In [21]:
# redact "Prefrontal Cortex" from region_level_2 for now
CLINPATH['region_level_2'] = CLINPATH['region_level_2'].replace({'Prefrontal Cortex':""})

# leave te APOE_e4_status as is for now . multiple are coded as "2,3" 
# but remove commas
CLINPATH["APOE_e4_status"] = CLINPATH["APOE_e4_status"].str.replace(",","")

# need to fix the path_autopsy_dx_main

In [22]:
CLINPATH[['sample_id', 'source_sample_id', 'time_from_baseline', 'GP2_id',
       'AMPPD_id']]

,sample_id,source_sample_id,time_from_baseline,GP2_id,AMPPD_id
0,HC01,hSDG07,,,
1,HC02,hSDG13,,,
2,HC03,hSDG101,,,
3,HC04,hSDG10,,,
4,HC05,hSDG30,,,
5,HC06,hSDG99,,,
6,PD01,hSDG151,,,
7,PD02,hSDG197,,,
8,PD03,hSDG208,,,
9,PD04,hSDG142,,,


In [23]:
CLINPATH['sample_id'].replace(subj_samp_mapper, inplace=True)
CLINPATH[['sample_id', 'source_sample_id', 'time_from_baseline', 'GP2_id',
       'AMPPD_id']]

/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_57365/2503552355.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  CLINPATH['sample_id'].replace(subj_samp_mapper, inplace=True)


,sample_id,source_sample_id,time_from_baseline,GP2_id,AMPPD_id
0,hSDG07,hSDG07,,,
1,hSDG13,hSDG13,,,
2,hSDG101,hSDG101,,,
3,hSDG10,hSDG10,,,
4,hSDG30,hSDG30,,,
5,hSDG99,hSDG99,,,
6,hSDG151,hSDG151,,,
7,hSDG197,hSDG197,,,
8,hSDG208,hSDG208,,,
9,hSDG142,hSDG142,,,


In [24]:
# CLINPATH = CLINPATH.rename(columns={"source_sample_id":"source_subject_id"})
# CLINPATH["subject_id"] = CLINPATH["source_subject_id"].str.replace("hSDG","hSDG0")
STUDY

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
0,Single-cell transcriptomic and proteomic analy...,Single nucleus RNA sequencing dataset of PD br...,We provide an extensive single cell analysis p...,TEAM-HAFLER,Le Zhang Lab,Le Zhang,Le.zhang@yale.edu,Le Zhang,Le Zhang,le.zhang@yale.edu,...,Nan,Doi: https://doi.org/10.1101/2022.02.14.480397,Nan,12,Prefrontal cortex,Late-stage PD and control postmortem brains,0000-0002-4860-831X,Nan,Nan,V1_08/27/2023


dump these extractions from the excel file as og

In [25]:
og_path = metadata_path / "og"
STUDY.to_csv(og_path / f"STUDY.csv", index=False)
SAMPLE.to_csv(og_path / f"SAMPLE.csv", index=False)
SUBJECT.to_csv(og_path / f"SUBJECT.csv", index=False)
PROTOCOL.to_csv(og_path / f"PROTOCOL.csv", index=False)
CLINPATH.to_csv(og_path / f"CLINPATH.csv", index=False)

## validate v1

QC these tables and make sure they are conformant


In [26]:
metadata_version = "v1"

METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

In [27]:
STUDY['metadata_version_date'] = METADATA_VERSION_DATE

In [28]:
CDE = CDEv1
tables = CDE['Table'].unique()
v1_path = metadata_path / "v1"

dfs = {}
for table in tables:
    df = read_meta_table(og_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table
    df.to_csv(v1_path / f"{table}.csv", index=False)


recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **4 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- DUA_version: 1/1 empty rows (REQUIRED)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- primary_diagnosis_text: 12/12 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **17 Fields with empty (NULL) values:**

	- source_RIN: 36/36 empty rows (REQUIRED)

	- RIN: 36/36 empty rows (REQUIRED)

	- file_description: 36/36 empty rows (REQUIRED)

	- header: 36/36 empty rows (REQUIRED)

	- annotation: 36/36 empty rows (REQUIRED)

	- preprocessing_reference

In [30]:
# make tables conform to CDE and save extra columns as "auxiliary"

for table in tables:
    df = dfs[table]
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v1_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v1_path / f"{table}.csv", index=False)
    

In [31]:
dfs['SAMPLE'][['sample_id','subject_id','source_sample_id','file_name','file_MD5']]

,sample_id,subject_id,source_sample_id,file_name,file_MD5
0,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_I1_001.fastq.gz,37912362469bc61a936dcc39a2b75138
1,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_R1_001.fastq.gz,b4d94bd92286d318de2b013ea44c5246
2,hSDG07,HC01,hSDG07,HSDG07HC_HHT_S6_L004_R2_001.fastq.gz,2a3c2c2303948f15d74b89ae4cbff409
3,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_I1_001.fastq.gz,1fbdbaba5e604aa852b9dc6af9e3dea4
4,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_R1_001.fastq.gz,3e14c63f5c55b4f29fcb0b1af5992924
5,hSDG101,HC03,hSDG101,hsDG101HC_HHT_S2_L002_R2_001.fastq.gz,f34b13c20fd03611b5772037bac11b37
6,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_I1_001.fastq.gz,7649f0db658f8d697b82a0ddc2dfedad
7,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_R1_001.fastq.gz,309aa23c50a13bb99ad88cdc4579d77d
8,hSDG10,HC04,hSDG10,HSDG10HC_HHT_S7_L004_R2_001.fastq.gz,187af38f53ba8b760e1da2e3fe602051
9,hSDG13,HC02,hSDG13,hsDG13HC_HHT_S1_L002_I1_001.fastq.gz,d3b2e5364b57bfa4a1f50166a9043949


## v1->v2

In [32]:
v2_path = metadata_path / "v2"

team_dataset_id = dataset_name.replace(" ", "_").replace("-", "_")

v2_tables, aux_tables = v1_to_v2(v1_path, v2_path, CDEv1, CDEv2, team_dataset_id=team_dataset_id)

# STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2 = v2_tables

recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int


In [33]:
v2_tables['STUDY']

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HAFLER,Le Zhang Lab,Single-cell transcriptomic and proteomic analy...,sn_rnaseq_pfc,Single nucleus RNA sequencing dataset of PD br...,We provide an extensive single cell analysis p...,Le Zhang,Le.zhang@yale.edu,Le Zhang,Le Zhang,...,NA,12,Prefrontal cortex,Late-stage PD and control postmortem brains,0000-0002-4860-831X,NA,NA,NA,v2.1_20241028,NA


### validate v2 tables


In [34]:
CDE = CDEv2
for table,df in v2_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()


recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **6 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ **15 Fields with empty (NULL) values:**

	- AMPPD_id: 12/12 empty rows (REQUIRED)

	- GP2_

## v2->v3

In [35]:
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

f"{v3_meta_tables}"

"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']"

In [36]:
v3_path = metadata_path / "v3"

v3_tables, aux_tables = v2_to_v3_PMDBS(v2_path, v3_path, CDEv2, CDEv3)

recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
recoding replicate_count as int
recoding repeated_sample as int


In [37]:
v3_tables.keys()

dict_keys(['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'PMDBS', 'ASSAY_RNAseq', 'CLINPATH', 'CONDITION'])

In [38]:
v3_tables['STUDY']

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HAFLER,Le Zhang Lab,Single-cell transcriptomic and proteomic analy...,sn_rnaseq_pfc,Single nucleus RNA sequencing dataset of PD br...,We provide an extensive single cell analysis p...,Le Zhang,Le.zhang@yale.edu,Le Zhang,Le Zhang,...,12,Prefrontal cortex,Late-stage PD and control postmortem brains,NA,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",0000-0002-4860-831X,NA,NA,v3.0_20241028,NA


### validate v3 tables


In [39]:
CDE = CDEv3
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    # should we write validation logs?

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **6 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)

	- preprocessing_references: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- github_url: 1/1 empty rows (REQUIRED)

	- protocols_io_DOI: 1/1 empty rows (REQUIRED)

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- primary_diagnosis_text: 12/12 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as 

In [40]:
v3_tables['CONDITION']

Field,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,healthy_control,Case-Control,Control,NA,NA
1,idiopathic_pd,Case-Control,Case,NA,NA


v3_tables['CONDITION]

In [41]:
STUDY = v3_tables['STUDY']
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HAFLER,Le Zhang Lab,Single-cell transcriptomic and proteomic analy...,sn_rnaseq_pfc,Single nucleus RNA sequencing dataset of PD br...,We provide an extensive single cell analysis p...,Le Zhang,Le.zhang@yale.edu,Le Zhang,Le Zhang,...,12,Prefrontal cortex,Late-stage PD and control postmortem brains,NA,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",0000-0002-4860-831X,NA,NA,v3.0_20241028,NA


In [42]:
v3_tables['SAMPLE']['condition_id']

0     healthy_control
1     healthy_control
2     healthy_control
3     healthy_control
4     healthy_control
5     healthy_control
6       idiopathic_pd
7       idiopathic_pd
8       idiopathic_pd
9       idiopathic_pd
10      idiopathic_pd
11      idiopathic_pd
Name: condition_id, dtype: object

-------------------------
## check md5s



In [43]:
print(team)

source = "pmdbs"

bucket = f"asap-raw-team-{team}-{source}-{dataset_name}"
bucket = f"asap-raw-data-team-{team}" # for now old locations


key_file_path = Path.home() / f"Projects/ASAP/{team}-credentials.json"

res = authenticate_with_service_account(key_file_path)
print(res)
prefix = "fastqs/**/*.gz"
bucket_files_md5 = get_md5_hashes( bucket, prefix)

hafler
CompletedProcess(args='gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hafler-credentials.json', returncode=0, stdout='', stderr='Activated service account credentials for: [raw-admin-hafler@dnastack-asap-parkinsons.iam.gserviceaccount.com]\n')
gsutil -u dnastack-asap-parkinsons hash -h gs://asap-raw-data-team-hafler/fastqs/**/*.gz


In [44]:

checksum = v3_tables['DATA'][['file_name','file_MD5']]
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)
checksum['check1'] = checksum['file_MD5']

checksum[checksum.check1 != checksum.check2].file_name.to_list()



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_57365/2789596456.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_57365/2789596456.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5']


[]

In [45]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()
#empty means success!!

[]

## Create metadata package


In [46]:
export_path = root_path / f"{team}"

create_upload_medadata_package(export_path, v3_tables)


In [47]:
schema = CDEv3[CDEv3["Table"] == "SAMPLE"]
schema

,Table,Field,Description,DataType,Required,Validation,Shared_key
40,SAMPLE,sample_id,Sample ID.: ID of each individual sample use...,String,Required,NaN,1.0
41,SAMPLE,subject_id,Subject ID.: Can be multiple samples from ea...,String,Required,NaN,1.0
42,SAMPLE,source_sample_id,Source Sample ID.: ID provided by Brain bank...,String,Required,NaN,1.0
43,SAMPLE,replicate,Replicate.: Indicator for replicates of the ...,String,Required,NaN,0.0
44,SAMPLE,replicate_count,Replicate Count.: Number of replicates of th...,Integer,Required,NaN,0.0
45,SAMPLE,repeated_sample,Repeated Sample.: Repeated Sample indicator....,Integer,Required,NaN,0.0
46,SAMPLE,batch,Batch.: Label to enumerate the batch. This w...,String,Required,NaN,1.0
47,SAMPLE,organism,Organism.: Organism of the subject. Currentl...,Enum,Required,"[""Human"", ""Mouse"",""Dog"",""Fly"",""Other""]",0.0
48,SAMPLE,tissue,"Tissue.: Type of tissue. e.g. ""Brain"" or ot...",String,Required,NaN,0.0
49,SAMPLE,assay_type,Type of Assay. Refers to the ASSAY_* table. Ex...,Enum,Required,"['scRNAseq','snRNAseq','bulkRNAseq']",0.0


In [48]:
SAMPLE = v3_tables['SAMPLE']
SUBJECT = v3_tables['SUBJECT']
SUBJECT

,subject_id,source_subject_id,biobank_name,sex,age_at_collection,race,primary_diagnosis,primary_diagnosis_text
0,HC01,hSDG07,NKI/NYUGSOM,Female,81.0,Black or African American,Healthy Control,NA
1,HC02,hSDG13,NKI/NYUGSOM,Female,65.0,White,Healthy Control,NA
2,HC03,hSDG101,NKI/NYUGSOM,Female,72.0,White,Healthy Control,NA
3,HC04,hSDG10,NKI/NYUGSOM,Male,70.0,White,Healthy Control,NA
4,HC05,hSDG30,NKI/NYUGSOM,Male,63.0,Black or African American,Healthy Control,NA
5,HC06,hSDG99,NKI/NYUGSOM,Male,69.0,White,Healthy Control,NA
6,PD01,hSDG151,NKI/NYUGSOM,Female,81.0,White,Idiopathic PD,NA
7,PD02,hSDG197,NKI/NYUGSOM,Female,72.0,White,Idiopathic PD,NA
8,PD03,hSDG208,NKI/NYUGSOM,Female,77.0,White,Idiopathic PD,NA
9,PD04,hSDG142,NKI/NYUGSOM,Male,79.0,White,Idiopathic PD,NA


______________